In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
dataset_2016 = 'Burlington_Stats_2016_Census.csv'
dataset_2021 = 'Burlington_Stats_2021_Census.csv'
Private_Schools = 'Private School Boards Ontario.csv'
Public_Schools = 'Public School Boards Ontario.csv'
data = pd.read_csv(dataset, on_bad_lines = 'skip', low_memory = True, engine = 'python')
data.sample(10)

,REF_DATE,GEO,DGUID,Investment status,Ownership type,Property type,Period of construction,Estimates,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
4482,2021,"Burlington, City",2016A00053524002,Non-investment property,"Total, all ownership types",Property with multiple residential units,2011 to 2015,Average assessment value,Dollars,81,units,0,v1484734519,387.4.1.8.9.3,NaN,..,NaN,NaN,0
1917,2021,"Burlington, City",2016A00053524002,Investment property,"Total, all ownership types",Semi-detached house,1971 to 1980,Median total living area,Square feet,279,units,0,v1484732094,387.2.1.3.4.7,1480.0,NaN,NaN,NaN,0
3376,2021,"Burlington, City",2016A00053524002,Investment property,Business and government,Other property type,2001 to 2005,Average assessment value,Dollars,81,units,0,v1484733553,387.2.3.7.7.3,NaN,..,NaN,NaN,0
5221,2021,"Burlington, City",2016A00053524002,Non-investment property,Business and government,Single-detached house,2006 to 2010,Median total living area,Square feet,279,units,0,v1484735398,387.4.3.2.8.7,NaN,..,NaN,NaN,0
4684,2021,"Burlington, City",2016A00053524002,Non-investment property,Person,Semi-detached house,1981 to 1990,Percentage - investment status,Percent,239,units,0,v1484734861,387.4.2.3.5.2,95.0,NaN,NaN,NaN,1
3029,2021,"Burlington, City",2016A00053524002,Investment property,Business and government,Single-detached house,2001 to 2005,Average total living area,Square feet,279,units,0,v1484733206,387.2.3.2.7.6,3800.0,NaN,NaN,NaN,0
98,2021,"Burlington, City",2016A00053524002,"Total, all investment property statuses","Total, all ownership types",Single-detached house,1971 to 1980,Number,Number,223,units,0,v1484730275,387.1.1.2.4.1,6985.0,NaN,NaN,NaN,0
1718,2021,"Burlington, City",2016A00053524002,"Total, all investment property statuses",Business and government,Property with multiple residential units,2011 to 2015,Median assessment value,Dollars,81,units,0,v1484731755,387.1.3.8.9.4,NaN,..,NaN,NaN,0
813,2021,"Burlington, City",2016A00053524002,"Total, all investment property statuses",Person,Row house,1961 to 1970,Percentage - investment status,Percent,239,units,0,v1484730990,387.1.2.4.3.2,100.0,NaN,NaN,NaN,1
2656,2021,"Burlington, City",2016A00053524002,Investment property,Person,Condominium apartment,2001 to 2005,Median assessment value,Dollars,81,units,0,v1484732833,387.2.2.5.7.4,310000.0,NaN,NaN,NaN,0
